# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import os
import re
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    path = 'names'
    ldirs = os.listdir(path)
    ldirs.sort()
    df = pd.concat([pd.concat([pd.Series(int(re.findall(r'\d+', file)[0])),
                               pd.read_csv(f'{path}/{file}', header = None)], 
                              axis = 1, ignore_index = True).ffill() 
                    for file in ldirs if file.endswith('.txt')], ignore_index = True)
    # note : une autre methode avec assign serait possible et plus courte
    df.rename({0:'year', 1:'name', 2:'gender', 3:'births'}, axis = 1, inplace = True)
    return df.astype({'year':'int16', 'name':'str', 'gender':'str', 'births':'int64'})

In [3]:
df = df_names_us()

In [4]:
df.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [5]:
df.dtypes

year       int16
name      object
gender    object
births     int64
dtype: object

In [6]:
df.index

RangeIndex(start=0, stop=2052781, step=1)

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [7]:
# names fr
def df_names_fr():
    # certaines personnes peuvent se prénommer Na
    df = pd.read_csv("nat2021_csv/nat2021.csv", sep=';', na_values = '', keep_default_na = False)
    # remap les valeurs de genre
    df['sexe'] = df['sexe'].map({1:'M', 2:'F'})
    # filtre les prénoms
    # attention NaN > 2 est False et les NaN posent problème avec | !
    df = df[(df['preusuel'].str.len() >= 2) & (df['preusuel'] != "_PRENOMS_RARES")]
    # aligne la casse des prénoms
    df['preusuel'] = df['preusuel'].str.title()
    # supprime les lignes inutiles
    df.dropna(how = 'any', inplace = True)
    df = df[df['annais'].str.match(r'\d+')]
    # aligne les noms de colonnes avec le df us
    df.rename({'sexe':'gender', 'preusuel':'name', 'annais':'year', 'nombre':'births'}, axis = 1, inplace = True)
    # trie selon l'ordre demandé
    df.sort_values(by = ['year', 'gender', 'births', 'name'], ascending = [True, True, False, True], inplace = True)
    df.reset_index(inplace = True)
    df = df[['year', 'name', 'gender', 'births']]
    df = df.astype({'year':'int16','name':'str', 'gender':'str', 'births':'int64'})
    return df

In [8]:
df_fr = df_names_fr()

In [9]:
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [10]:
df_fr.dtypes

year       int16
name      object
gender    object
births     int64
dtype: object

In [11]:
df_fr.shape

(648330, 4)

In [12]:
# le cas particulier
df_fr[df_fr['name'] == "Na"]

,year,name,gender,births
404624,2003,Na,F,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [13]:
# taux de change
def df_taux_change(devises):
    # charge les données en prenant la ligne des Titres en header
    df = pd.read_csv("Webstat_Export.csv", sep=';', header = 0 , skiprows = [1, 2, 3, 4, 5], decimal = ',', na_values = '-')
    # indexe sur les dates et trie
    df.rename({'Titre :':'date'}, axis = 1, inplace = True)
    df['date'] = pd.to_datetime(df['date'], format = "%d/%m/%Y")
    df.set_index(keys = 'date', inplace = True)
    df.sort_index(inplace = True)
    # restreint les colonnes aux codes ISO3
    cols = df.columns[df.columns.str.contains("\([A-Z]{1,3}\)$", regex = True)]
    df = df[cols]
    df.columns = cols.str.slice(start = -4, stop = -1)
    # drop les lignes contenant uniquement des na
    df.dropna(how = 'all', inplace = True)
    return df[devises]

In [14]:
df_taux_change(["CHF", "GBP", "USD"])

,CHF,GBP,USD
date,,,
1999-01-04,1.6168,0.71110,1.1789
1999-01-05,1.6123,0.71220,1.1790
1999-01-06,1.6116,0.70760,1.1743
1999-01-07,1.6165,0.70585,1.1632
1999-01-08,1.6138,0.70940,1.1659
...,...,...,...
2022-09-14,0.9612,0.86498,0.9990
2022-09-15,0.9572,0.86934,0.9992
2022-09-16,0.9579,0.87400,0.9954


### Tests

In [15]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [16]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [17]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 9.688s

OK
